<a href="https://colab.research.google.com/github/HeleneFabia/nlp-exploration/blob/main/notebooks/nlp_with_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face NLP Course



In [ ]:
# install libraries
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install transformers[sentencepiece]
!pip install sacrebleu

## 1. Transformer models

### What can Transformers do?

Playing around with HuggingFace's OTB models:

In [ ]:
# imports
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("The ocean is beautiful.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9998816251754761}]

In [ ]:
generator = pipeline("text-generation")
generator("Looking at the ocean in front of me, I felt")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Looking at the ocean in front of me, I felt like an airplane flying right.\n\nThe storm had descended very quickly, so it appeared this was about to fall at a later date.\n\nWe all started feeling a wave of panic:'}]

In [ ]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="What is my hobby?",
    context="I work as an engineer, but in my free time I enjoy cooking."
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'answer': 'cooking', 'end': 58, 'score': 0.428894579410553, 'start': 51}

### How do Transformers work?

Important concepts: 
- **self-supervised learning**: labels are automatically computed from the input
- **pretraining**: training a model from scratch on very large amounts of data
- **transfer learning**: fine-tuning a pretrained model in a supervised manner with a smaller dataset for a specific language task
- **encoder**: receives input and builds representation of it (optimized for  acquiring an understanding from inputs)
- **decoder**: receives encoder's representation plus other inputs in order to generate a target sequence (optimized for generating an output)
- **encoder-only models** (e.g., BERT, DistilBERT): for tasks that require understanding of the input e.g., sentence classification, named entity recognition
- **decoder-only models** (e.g., GPT, GPT-2): for generative tasks e.g., text generation
- **encoder-decoder /seq2seq models** (e.g., BART, Marian): for generative tasks that require an input e.g., translation, summarization
- **attention layer**: tells the model to pay attention to specific words in the input


## 2. Using HuggingFace Transformers

### Simple pipeline

Tokenizer:
- splits the input words/subwords/symbols (=tokens), since a model cannot process words directly
- maps each token to an integer
- adds additional inputs necessary for the model
- tokenization needs to happen in exactly the same way as was done with the data used for pretraining a model

In [ ]:
# imports
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)

from torch.nn.functional import softmax

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
input = ["I am looking at the ocean. How beautiful!"]
tokenized_input = tokenizer(input, padding=True, truncation=True)
print(tokenized_input)

{'input_ids': [[101, 1045, 2572, 2559, 2012, 1996, 4153, 1012, 2129, 3376, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
tokenized_input["input_ids"] = torch.tensor(tokenized_input["input_ids"])
tokenized_input["attention_mask"] = torch.tensor(tokenized_input["attention_mask"])
print(tokenized_input)

{'input_ids': tensor([[ 101, 1045, 2572, 2559, 2012, 1996, 4153, 1012, 2129, 3376,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


input vector:
- of shape (batch_size, sequence_length, hidden_size)
- batch_size: number of sequences per batch
- sequence_length: length of numerical representation of sequence
- hidden_size:  vector dimension of each model input (depends on the model)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
output = model(**tokenized_input)
print(output.logits)
prediction = softmax(output.logits, dim=1)
print(prediction)

tensor([[-4.3032,  4.5966]], grad_fn=<AddmmBackward0>)
tensor([[1.3640e-04, 9.9986e-01]], grad_fn=<SoftmaxBackward0>)


In [ ]:
input_id = 0
class_prediction = int(torch.argmax(prediction))
print(f"Prediction for sentiment of '{input[input_id]}':", 
      model.config.id2label[class_prediction],
      f"with {prediction.tolist()[input_id][class_prediction]:.4f}% probability"
      )

Prediction for sentiment of 'I am looking at the ocean. How beautiful!': POSITIVE with 0.9999% probability


### Models

In [ ]:
# imports
from transformers import (
    BertConfig,
    BertModel,
)

In [ ]:
config = BertConfig()
print(config)
model = BertModel(config)  # randomly initialized
model = BertModel.from_pretrained("bert-base-cased")  # pretrained (https://huggingface.co/bert-base-cased)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.save_pretrained("/path/to/my_trained_model")

### Tokenizers

Used to transform language data into numerical data so that te model can process it. Some approach are:

**Word-based tokenizer**
- split raw text into words and find numerical representation for them
- would need A LOT of different input IDs (one for each word in a language) 
- no means of showing relationships between words ("dog" and "dogs" would have different input IDs)
- need "unknown" token ("[UNK]") for words that are not in the vocabulary.

**Character-based tokenizer**
- raw text is split into characters
- fewer distinct input IDs are necessary but numerical sequences would be much longer with this approach

**Subword tokenizer**
- frequently used words remain as they are, less frequently used ones are split into meaningful subwords (e.g., "annoyingly" --> "annoying" + "ly")
- good tradeoff between small number of distinct input IDs and short sequences
- examples: WordPiece (BERT), BPE (GPT-2), and Unigram

In [ ]:
from transformers import (
    BertTokenizer,
    AutoTokenizer
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# essentially the same, but the second module is a wrapper that can be used with any checkpoint

In [ ]:
tokenizer("The sea is incredibly blue and glittering today.")

{'input_ids': [101, 1109, 2343, 1110, 12170, 2221, 1105, 22837, 2052, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

**encoding** = general process of converting text to numbers

tokenization = splitting text into tokens (according to the way it was done for the pretrained model we want to use)

In [ ]:
tokens = tokenizer.tokenize("The sea is incredibly blue and glittering today.")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['The', 'sea', 'is', 'incredibly', 'blue', 'and', 'glittering', 'today', '.']
[1109, 2343, 1110, 12170, 2221, 1105, 22837, 2052, 119]


**decoding** = converting numbers to text

In [ ]:
text = tokenizer.decode(ids)
print(text)

The sea is incredibly blue and glittering today.


### Handling multiple sequences

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
sequence = "Hmmm, I love green tea!"
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input = torch.tensor([ids]) # model expects a batch, not one single sample
print(input.shape)

output = model(input)
print(output.logits)

torch.Size([1, 8])
tensor([[-2.6319,  2.8690]], grad_fn=<AddmmBackward0>)


**padding** = making sure all sequences have the same length by adding a padding token

In [ ]:
sequences = ["Hmmm, I love green tea!", "It's Thursday"]  # sequences are of different length!
try:
  inputs = tokenizer(sequences, return_tensors="pt")
  input = inputs["input_ids"]
  print(input.shape)
  output = model(input)
  print(output.logits)
except ValueError as error:
  print(error)


Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.


**attention mask** = tensor of exact same shape as input IDs; filled with 0s and 1s; 1 means a specific token is paid attention to in the attention layer, 0 means it is not paid attention to

In [ ]:
input_ids = [[5, 5, 5], [5, 5, tokenizer.pad_token_id]]
attention_mask = [[1, 1, 1], [1, 1, 0]]

outputs = model(torch.tensor(input_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 0.8322, -0.7892],
        [ 0.3235, -0.2539]], grad_fn=<AddmmBackward0>)


**truncation** = limiting the length of a sequence
(necessary because models can only handle up to 512/1024 tokens per sequence - however, there are also models (e.g. Longformer) which can handle longer sequences)

In [ ]:
inputs = tokenizer(
    sequences, 
    padding=True, 
    truncation=True,
    return_tensors="pt"  # or "np" for numpy arrays
    )

**special tokens** = added to the inputs, for example [CLS] (= beginning of a sequence) and [SEP] (= end of a sequence)

In [ ]:
print(inputs["input_ids"][0])
print(tokenizer.decode(inputs["input_ids"][0]))

tensor([  101, 17012,  2213,  1010,  1045,  2293,  2665,  5572,   999,   102])
[CLS] hmmm, i love green tea! [SEP]


## 3. Fine-tuning a pretrained model

### Processing the data

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

In [ ]:
ds = load_dataset("glue", "mrpc", split="train")
print("Columns:", ds.column_names)
print("Number of samples:", len(ds))
print("Classes:", ds.features["label"].names)
print("Example:", ds[0])

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


Columns: ['sentence1', 'sentence2', 'label', 'idx']
Number of samples: 3668
Classes: ['not_equivalent', 'equivalent']
Example: {'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

**token type ids** = in this example, the tensor tells the model which input ids belong to the first sentence and which belong to the second sentence.

In [ ]:
inputs = tokenizer(ds["sentence1"][10], ds["sentence2"][10])
print(inputs["input_ids"])
print(inputs["token_type_ids"])

[101, 6094, 2437, 2009, 6211, 2005, 10390, 2000, 22505, 2037, 13930, 1999, 10528, 2457, 2180, 10827, 2160, 6226, 1999, 2233, 1012, 102, 6094, 2437, 2009, 6211, 2005, 10390, 2000, 22505, 2037, 13930, 1999, 10528, 2457, 2180, 26203, 1010, 2160, 6226, 1999, 2233, 1998, 2001, 11763, 2011, 1996, 2317, 2160, 1012, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
tokenized_ds = tokenizer(
    ds["sentence1"],
    ds["sentence1"],
    padding=True,
    truncation=True
)  # will require a lot of RAM; will return the dataset as a dictionary

In [ ]:
def tokenize_func(dataset):  # use with Dataset.map() method
  return tokenizer(dataset["sentence1"], dataset["sentence2"], truncation=True)
  # no padding, since whole dataset would be padded to the same length (unnecessary)
  # instead, padding is applied to each batch 

In [ ]:
ds = load_dataset("glue", "mrpc")
print(ds)
tokenized_ds = ds.map(tokenize_func, batched=True)
print(tokenized_ds)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})


**dynamic padding** = apply padding in the collate function that builds the DataLoader

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_ds["train"][:6]
print([len(sample) for sample in samples["input_ids"]]) # sequences are still of different lengths

[50, 59, 47, 67, 59, 50]


In [ ]:
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
batch = data_collator(samples)  # automatically padded to max length in whole dataset
print({k: v.shape for k, v in batch.items()})  

{'attention_mask': torch.Size([6, 67]), 'input_ids': torch.Size([6, 67]), 'token_type_ids': torch.Size([6, 67]), 'labels': torch.Size([6])}


### Fine-tuning a model with the Trainer API

In [ ]:
import numpy as np

from transformers import (
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer
    )
from datasets import load_metric

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
def compute_metrics(valid_preds):
  metric = load_metric("glue", "mrpc"),
  logits, labels = valid_preds
  preds_cls = np.argmax(logits, axis=-1)
  return metric.compute(predictions=preds_cls, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

### A full training

In [ ]:
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AdamW,
    get_scheduler
)
from datasets import load_metric
# from accelerate import Accelerator

In [ ]:
tokenized_ds = tokenized_ds.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format("torch")

In [ ]:
train_dl = DataLoader(
    tokenized_ds["train"], 
    shuffle=True, 
    batch_size=8,
    collate_fn=data_collator
    )

valid_dl = DataLoader(
    tokenized_ds["validation"], 
    batch_size=8, 
    collate_fn=data_collator
    )

In [ ]:
for batch in train_dl:  # padding is applied to max length of respective batch
  print({k: v.shape for k, v in batch.items()})
  break

{'attention_mask': torch.Size([8, 72]), 'input_ids': torch.Size([8, 72]), 'labels': torch.Size([8]), 'token_type_ids': torch.Size([8, 72])}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# accelerator = Accelerator()

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# train_dl, valid_dl, model, optimizer = accelerator.prepare(
#    train_dl, 
#    valid_dl,
#    model,
#    optimizer
#)  # --> handles device placement so putting model and data on device during training is unnecessary when working with Accelerator

In [ ]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
  for batch in train_dl:
    batch = {k: v.to(device) for k, v in batch.items()}
  
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    # accelerator.backward(loss)
    
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    
    progress_bar.update(1)

In [ ]:
metric = load_metric("glue", "mrpc")
model.eval()
for batch in valid_dl:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

## 4. Datasets library

In [ ]:
from datasets import (
    load_dataset_builder, 
    load_dataset,
)
from transformers import (
    BertTokenizerFast,
    AutoTokenizer
)

import torch
from torch.utils.data import(
    DataLoader
)

In [ ]:
dataset_builder = load_dataset_builder(path="poem_sentiment")

train_dataset = load_dataset(path="poem_sentiment", split="train")
# valid_dataset = load_dataset(path="poem_sentiment", split="validation")
# test_dataset = load_dataset(path="poem_sentiment", split="test")

In [ ]:
print("Description:", train_dataset.description)
print("Num data entries:", len(train_dataset))
print("Column names:", train_dataset.column_names)
print("Classes:", train_dataset.features["label"].names)
print("Example data entry:", train_dataset[0])

Description: Poem Sentiment is a sentiment dataset of poem verses from Project Gutenberg. This dataset can be used for tasks such as sentiment classification or style transfer for poems.

Num data entries: 892
Column names: ['id', 'verse_text', 'label']
Classes: ['negative', 'positive', 'no_impact', 'mixed']
Example data entry: {'id': 0, 'verse_text': 'with pale blue berries. in these peaceful shades--', 'label': 1}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
train_enc_ds = train_dataset.map(lambda examples: 
                                          tokenizer(
                                              examples["verse_text"], 
                                              truncation=True,
                                              padding="max_length",
                                          ),
                                 batched=True
                                 )

In [ ]:
print("Column names of encoded dataset:", train_enc_ds.column_names)
print("Tokenized data entry:", train_enc_ds[0])

Column names of encoded dataset: ['attention_mask', 'id', 'input_ids', 'label', 'token_type_ids', 'verse_text']
Tokenized data entry: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# use dataset with pytorch
train_enc_ds.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])

# create pytorch data loader
train_dl = DataLoader(train_enc_ds, batch_size=32)
next(iter(train_dl))

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,  1114,  4554,  ...,     0,     0,     0],
         [  101,  1122,  5611,  ...,     0,     0,     0],
         [  101,  1105,  1115,  ...,     0,     0,     0],
         ...,
         [  101,  1106,  1115,  ...,     0,     0,     0],
         [  101,   192,  2386,  ...,     0,     0,     0],
         [  101,  1123, 15219,  ...,     0,     0,     0]]),
 'label': tensor([1, 2, 0, 3, 3, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 2, 1,
         2, 2, 2, 2, 2, 2, 2, 2]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])}

## 5. Tokenizers library

When training a model from scratch, it also makes sense to train the tokenizer from scratch. (Training a model from scratch makes sense if a model is not available in a particular language or if your corpus is very different from those other models were trained on). So whenever you want to pretrain a model and the dataset is different from the on used by an existing pretraiend model, you have to train a new tokenizer.

### Training a new tokenizer from an old one

A tokenizer needs to be trained on a corpus in order to identify subwords that are of interest and occur most frequently in the corpus. Training a tokenizer is unlike training a model - it's a statistical process with the exact rules depending on the tokenization algorithm. It's not random (like model) but deterministic.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
raw_ds = load_dataset("code_search_net", "python")

In [ ]:
print(raw_ds["train"])
print(raw_ds["train"][1]["whole_func_string"])

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})
def close(self):
        '''
        Cleanly shutdown the router socket
        '''
        if self._closing:
            return
        log.info('MWorkerQueue under PID %s is closing', os.getpid())
        self._closing = True
        # pylint: disable=E0203
        if getattr(self, '_monitor', None) is not None:
            self._monitor.stop()
            self._monitor = None
        if getattr(self, '_w_monitor', None) is not None:
            self._w_monitor.stop()
            self._w_monitor = None
        if hasattr(self, 'clients') and self.clients.closed is False:
            self.clients.close()
        if hasattr(self, 'workers') and self.workers.closed is False:
            self.workers.close()
        if ha

It makes sense converting the dataset to an iterator (e.g., a list of lists of texts), so that the tokenizer can train on batches of texts AND the whole dataset does not need to be loaded into memory all at once.

In [ ]:
# training_corpus = [
#                   raw_ds["train"][i: i + 1000]["whole_func_string"] 
#                   for i in range(0, len(raw_ds["train"]), 1000)
#                   ]
# creates a list of lists of 1000 texts each and load it into memory

# use parentheses instead of brackets to create a Python generator that does not
# load anything into memory until it's necessary.
# nothing is loaded into memory but object is created that can be used in Python
# for loop - however, the object can only be used once

training_corpus = (
    raw_ds["train"][i: i + 1000]["whole_func_string"] 
    for i in range(0, len(raw_ds["train"]), 1000)
)

In [ ]:
gen1 = (i for i in range(10))
gen2 = [i for i in range(10)]
print(gen1)
print(gen2)

<generator object <genexpr> at 0x7f8cf2e11b50>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
def get_training_corpus():
  return (
      raw_ds["train"][i: i + 1000]["whole_func_string"]
      for i in range(0, len(raw_ds["train"]), 1000)
  )

# ALTERNATIVE: use yield statement to create generator
# def get_training_corpus():
#  for start_idx in range(0, len(raw_ds["train"]), 1000):
#    yield raw_ds["train"][start_idx: start_idx + 1000]["whole_func_string"] 

training_corpus = get_training_corpus()

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
# not starting *entirely* from scratch but using tokenization algorithm of GPT-2

In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

print(old_tokenizer.tokenize(example))

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, vocab_size=52000)

In [ ]:
print(tokenizer.tokenize(example))

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


### Normalization and pre-tokenization

Before splitting a text into subtokens, a tokenizer performs normalization and pre-tokenization before.

**normalization** = general cleaning e.g., removing unnecessary whitespaces, lowercasing, removing accents

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Hellö, how ARE YOU tòday?"))

hello, how are you today?


**pre-tokenization** = splitting the text into smaller entities i.e., words (e.g., at whitespaces or punctuation) (which are later split into tokens).

In [ ]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("hello, how are you today?"))

[('hello', (0, 5)), (',', (5, 6)), ('how', (7, 10)), ('are', (11, 14)), ('you', (15, 18)), ('today', (19, 24)), ('?', (24, 25))]


### Byte-Pair Encoding (BPE) tokenization

- used for training GPT models

Training of tokenizer:
- first, normalization and pre-tokenization are applied
- computes unique set of words and builds vocabulary by taking all the symbols used to write those words
  - corpus: `["hug", "pug", "pun", "bun", "hugs"]`
  - initial vocabulary: `["b", "g", "h", "n", "p", "s", "u"]` 
- vocabulary is expanded by learning merges i.e., merging two elements of existing vocabulary - as such, longer and longer subwords are added to the vocabulary
  - this is done by looking by the most frequent "pair" i.e., the most frequently appearing merge between elements in the existing vocabulary. The most frequent pair will be merged and added to the vocabulary. Then the processed is repeated.
  - vocabulary during merging: `["b", "g", "h", "n", "p", "s", "u", "ug"]` --> `["b", "g", "h", "n", "p", "s", "u", "ug", "un"]` --> `["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]` --> ...

Tokenization with trained tokenizer:
- normalization
- pre-tokenization
- splitting words into individual characters
- applying merge rules learned in order to those splits
- example: `"unhug"` will be tokenized as `["un", "hug"]`, `"bug"` as `["[UNK]", "ug"]` (since "b" was not in the corpus with which tokenizer was trained)

### WordPiece tokenization

- starts like BPE from base vocabulary and then learns merging rules
- does not select most frequent pairs (like BPE) but calculates special score for each pair
- does not save the rules learned from training (like BPE) but saves only the final vocabulary

### Unigram tokenization

- starts from a big vocabulary and removes tokens from it until it reaches a desired vocabulary size
computes loss over the corpus given the current vocabulary and iterativels removes the 10-20% of tokens based on how little the loss would increase by their removal



### Building a tokenizer, block by block

Tokenization consists of 
- normalization
- pre-tokenization
- tokenizer model (like BPE, WordPiece, ...)
- post-processing (adding special tokens, generating attention mask and token type IDs)

In [ ]:
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)
from transformers import (
    PreTrainedTokenizerFast,
    BertTokenizerFast
)

In [ ]:
ds = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

In [ ]:
def get_training_corpus():
  for i in range(0, len(ds), 1000):
    yield ds[i: i + 1000]["text"]

In [ ]:
# model block
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [ ]:
# normalizer block 
tokenizer.normalizer = normalizers.Sequence(
    [
     normalizers.NFD(),  # NFD Unicode normalizer
     normalizers.Lowercase(),
     normalizers.StripAccents()
    ]
)
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [ ]:
# pre-tokenizer block
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace() # splits of whitespaces and punctuation (excluding underscore character)
print(tokenizer.pre_tokenizer.pre_tokenize_str("Today's such a beautiful and sunny day."))

[('Today', (0, 5)), ("'", (5, 6)), ('s', (6, 7)), ('such', (8, 12)), ('a', (13, 14)), ('beautiful', (15, 24)), ('and', (25, 28)), ('sunny', (29, 34)), ('day', (35, 38)), ('.', (38, 39))]


In [ ]:
# train tokenizer
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
encoding = tokenizer.encode("Let's test the tokenizer, shall we?")
print("Tokens:", encoding.tokens)
print("Input IDs:", encoding.ids)
print("Offsets:", encoding.offsets)
print("Attention mask:", encoding.attention_mask)
print("Special token mask:", encoding.special_tokens_mask)
print("Overflowing:", encoding.overflowing)

Tokens: ['let', "'", 's', 'test', 'the', 'tok', '##eni', '##zer', ',', 'shall', 'we', '?']
Input IDs: [3019, 11, 61, 3611, 1333, 24319, 18903, 6614, 16, 11448, 1626, 35]
Offsets: [(0, 3), (3, 4), (4, 5), (6, 10), (11, 14), (15, 18), (18, 21), (21, 24), (24, 25), (26, 31), (32, 34), (34, 35)]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Special token mask: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Overflowing: []


In [ ]:
# post-processing
cls_token_id = tokenizer.token_to_id("[CLS]") # to add at beginning of sequence
sep_token_id = tokenizer.token_to_id("[SEP]") # to add at end of each sentence in sequence
print("[CLS]", cls_token_id, "[SEP]", sep_token_id)

tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0", # token type IDs for single sentences ($A means single sentence)
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1", # tokent type IDs to use for sentence pairs ($B)
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

[CLS] 2 [SEP] 3


In [ ]:
encoding = tokenizer.encode("Let's test the tokenizer, shall we?")
print("Tokens:", encoding.tokens)
print("Input IDs:", encoding.ids)
print("Offsets:", encoding.offsets)
print("Attention mask:", encoding.attention_mask)
print("Special token mask:", encoding.special_tokens_mask)
print("Overflowing:", encoding.overflowing)

Tokens: ['[CLS]', 'let', "'", 's', 'test', 'the', 'tok', '##eni', '##zer', ',', 'shall', 'we', '?', '[SEP]']
Input IDs: [2, 3019, 11, 61, 3611, 1333, 24319, 18903, 6614, 16, 11448, 1626, 35, 3]
Offsets: [(0, 0), (0, 3), (3, 4), (4, 5), (6, 10), (11, 14), (15, 18), (18, 21), (21, 24), (24, 25), (26, 31), (32, 34), (34, 35), (0, 0)]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Special token mask: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Overflowing: []


In [ ]:
# decoder
tokenizer.decoder = decoders.WordPiece(prefix="##")

tokenizer.decode(encoding.ids)

"let's test the tokenizer, shall we?"

In [ ]:
# save tokenizer
tokenizer.save("tokenizer.json")

#load tokenizer
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [ ]:
# wrap tokenizer in PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json",  # alternatively, load from file 
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)

Building a BPE (GPT-2) tokenizer from scratch:

In [ ]:
# choose model
tokenizer = Tokenizer(models.BPE())

In [ ]:
# add pre-tokenizer (not normalizer needed for GPT-2 tokenizer)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False) # add_prefix_space add space at beginning of sentence

In [ ]:
# train tokenizer
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
# post-processing
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False) # trim_offset means that start of offset will point at first character of word, not at space before it


In [ ]:
# decoder
tokenizer.decoder = decoders.ByteLevel()

In [ ]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>"
)
# OR
# wrapped_tokenizer = GPT2TokenizerFast(tokenier_object=tokenizer)

## 6. Main NLP tasks

### Token classification

- **Named entity recognition (NER)**: find names, locations and organization in a sentence; assign class to each token
- **Part-of-speech tagging (POS)**: mark each word as its respective par of speech (e.g., noun, verb,...)
- **Chunking**: find tokens that belong to the same entity

### Fine-tuning a masked language model

- pre-trained model can be fine-tuned on your data if the dataset is not too different from the corpus used from pretraining the model
- **domain adaptation**: on example of fine-tuning a model, namely on a dataset of special domain texts

In [ ]:
import math
from tqdm.auto import tqdm
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    default_data_collator,
    get_scheduler,
    pipeline
)
from datasets import load_dataset
from accelerate import Accelerator
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [ ]:
checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# how pretrained model works 
text = "This is a great [MASK]."

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
mask_token_idx = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_idx, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
  print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

This is a great deal.
This is a great success.
This is a great adventure.
This is a great idea.
This is a great feat.


In [ ]:
imdb_ds = load_dataset("imdb")

In [ ]:
print(imdb_ds["train"]["text"][0])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:
def tokenize_func(examples):
  result = tokenizer(examples["text"])
  if tokenizer.is_fast:
    result["word_ids"] = [result.word_ids(i) 
    for i in range(len(result["input_ids"]))]
  return result

In [ ]:
tokenized_ds = imdb_ds.map(
    tokenize_func, 
    batched=True,
    remove_columns=["text", "label"]
    )

In [ ]:
print(tokenized_ds)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 50000
    })
})


In [ ]:
print(tokenized_ds["train"]["input_ids"][0])
print(tokenized_ds["train"]["word_ids"][0])

[101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 3836, 1010, 19846

In [ ]:
print(tokenizer.model_max_length) # max sequence length the model can take

tokenized_samples = tokenized_ds["train"][6:9]
for idx, sample in enumerate(tokenized_samples["input_ids"]):
  print(f"Review {idx}: length {len(sample)}")
# some examples are longer than max length
# solution: concatenate all examples and then split them into chunks of equal size

512
Review 0: length 143
Review 1: length 388
Review 2: length 720


In [ ]:
# concatenate the samples 
concatenated_examples = {
    k: [i for sample in tokenized_samples[k] for i in sample] for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print("Concatenated reviews length:", total_length)

Concatenated reviews length: 1251


In [ ]:
# split the concatenated sample into chunks
chunk_size = 128 
chunks = {
    k: [v[i: i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, v in concatenated_examples.items()
} 
for chunk in chunks["input_ids"]:
  print("Chunk length:", len(chunk))

Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 128
Chunk length: 99


In [ ]:
# wrap the all of the above code in a function

def group_texts(examples):
  concatenated_examples = {
      k: [i for sample in examples[k] for i in sample] for k in examples.keys()
      }
  total_length = len(concatenated_examples["input_ids"])
  total_length = (total_length // chunk_size) * chunk_size
  result = {
      k: [v[i: i + chunk_size] for i in range(0, total_length, chunk_size)]
      for k, v in concatenated_examples.items()
  }
  # labels are the same as input_ids 
  # (since model will predict randomly masked tokens of input)
  result["labels"] = result["input_ids"].copy()  
  return result

In [ ]:
lm_ds = tokenized_ds.map(group_texts, batched=True)

In [ ]:
print(lm_ds)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 122957
    })
})


In [ ]:
print(tokenizer.decode(lm_ds["train"][3]["input_ids"]))
print(tokenizer.decode(lm_ds["train"][3]["labels"]))
# labels still need to be randomly masked (done in the data collator)         

' t matter what one's political views are because this film can hardly be taken seriously on any level. as for the claim that frontal male nudity is an automatic nc - 17, that isn't true. i've seen r - rated films with male nudity. granted, they only offer some fleeting views, but where are the r - rated films with gaping vulvas and flapping labia? nowhere, because they don't exist. the same goes for those crappy cable shows : schlongs swinging in the breeze but not a clitoris in sight. and those pretentious indie movies like
' t matter what one's political views are because this film can hardly be taken seriously on any level. as for the claim that frontal male nudity is an automatic nc - 17, that isn't true. i've seen r - rated films with male nudity. granted, they only offer some fleeting views, but where are the r - rated films with gaping vulvas and flapping labia? nowhere, because they don't exist. the same goes for those crappy cable shows : schlongs swinging in the breeze but n

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
# mlm_probability = fraction of tokens to mask

In [ ]:
# data collator expects list of dicts (each dict is one sample)
samples = [lm_ds["train"][i] for i in range(2)]
for sample in samples:
  _ = sample.pop("word_ids")
for chunk in data_collator(samples)["input_ids"]:
  print(tokenizer.decode(chunk))

[CLS] i rented i am hari - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs ifப ever tried [MASK] [MASK] [MASK] country [MASK] therefore being a fan of films considered " controversial " i really had ð see this for [MASK]. [MASK] [MASK] / > < br / > the plot [MASK] centered around a young swedish drama student named lena who wants [MASK] [MASK] everything she can about life. in particular she wants to focus her attentions to making [MASK] [MASK] [MASK] documentary on what the average sw ј thought about certain political issues [MASK]
[MASK] the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about [MASK] [MASK] on politics, she has sex with her drama teacher, classmates, and married men. < [MASK] / > < br / > what kills me about i am curious - yellow is that 40 years l, this was considered pornograph

In [ ]:
# downsample dataset
train_size = 10000
test_size = int(0.1 * train_size)
downsampled_ds = lm_ds["train"].train_test_split(
    train_size=train_size,
    test_size=test_size,
    seed=42
)
print(downsampled_ds)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 1000
    })
})


In [ ]:
batch_size = 64
logging_steps = len(downsampled_ds["train"]) // batch_size
model_name = checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps, # to track training loss each epoch
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_ds["train"],
    eval_dataset=downsampled_ds["test"],
    data_collator=data_collator
)

In [ ]:
trainer.train()

Evaluation

- good model assigns high probability to correct sentences and low probability to incorrect sentences
- one way of valutaing model is to look at probability the model assign to the next word in each sentence - if the probabilities are high, it means the model is not "perplexed" by the unseen examples --> measuring performance with exponential cross-entropy loss (= perplexity)

In [ ]:
eval_results = trainer.evaluate()

### Translation

= sequence-to-sequence task

Approach of translation can also be adapted to style transfer or generatie question answering.

Here, we will fine-tune a Marian model that was pre-trained from English to French.

In [ ]:
import numpy as np
from tqdm.auto import tqdm
from datasets import (
    load_dataset,
    load_metric
)
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AdamW,
    get_scheduler
)
from accelerate import Accelerator
from huggingface_hub import (
    notebook_login,
    Repository,
    get_full_repo_name
    )
import torch
from torch.utils.data import DataLoader

In [ ]:
# load dataset
raw_ds = load_dataset("kde4", lang1="en", lang2="fr")

In [ ]:
print(raw_ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})


In [ ]:
# split dataset into train and validation
split_ds = raw_ds["train"].train_test_split(train_size=0.9, seed=20)
split_ds["validation"] = split_ds.pop("test")

In [ ]:
print(split_ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})


In [ ]:
# dataset
print("EN:", split_ds["train"][1]["translation"]["en"])
print("FR (dataset):", split_ds["train"][1]["translation"]["fr"])

EN: Default to expanded threads
FR (dataset): Par défaut, développer les fils de discussion


In [ ]:
checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=checkpoint)
print("FR (model output):", translator("Default to expanded threads")[0]["translation_text"])

FR (model output): Par défaut pour les threads élargis


In the dataset, all English words are translated to French, whereas the pretrained model was trained on a corpus where some technical English words were used in the French translation - the goal of fine-tuning is to let the model learn the translation of these technical English words

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, return_tensors="pt")

In [ ]:
# inputs are processed normally
# tokenizer for targets is wrapped inside context manager with `as_target_tokenizer()`
# otherwise target are tokenized by input tokenizer (used for English!!).
en_sentence = split_ds["train"][1]["translation"]["en"]
fr_sentence = split_ds["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence)
incorrect_targets = tokenizer(fr_sentence)
with tokenizer.as_target_tokenizer():
  targets = tokenizer(fr_sentence)

print("Incorrect:", tokenizer.convert_ids_to_tokens(incorrect_targets["input_ids"]))
print("Correct:", tokenizer.convert_ids_to_tokens(targets["input_ids"]))

Incorrect: ['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
Correct: ['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [ ]:
max_input_length = 128
max_target_length = 128

def preprocess_func(examples):
  inputs = [ex["en"] for ex in examples["translation"]]
  targets = [ex["fr"] for ex in examples["translation"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  # put labels inside inputs dictionary
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_ds = split_ds.map(
    preprocess_func,
    batched=True,
    remove_columns=split_ds["train"].column_names
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
# data collator with padding for dynamic batching
# DataCollatorWithPadding would only pad input, but labels also need to be padded
# target should be padded with -100 so that these padding tokens are ignored in loss computation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_ds["train"][i] for i in range(1, 3)])
print(batch)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'input_ids': tensor([[47591,    12,  9842, 19634,     9,     0, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
         28149,   139, 33712, 25218,     0]]), 'labels': tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]]), 'decoder_input_ids': tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])}


Traditional metric used in translation: BLEU score (= how close is translation to label?) - uses statistical tools to measure model's output (instead of intelligibility of grammatical correctness)
Today, most commonly used score is SacreBLEU (since BLEU expects text already to be tokenized, which is a problem since different models use different tokenizers).

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
predictions = ["This plugin lets you translate web pages between several languages automatically."]
references = [["This plugin allows you to automatically translate web pages between several languages."]]

print("SacreBLEU score:", metric.compute(predictions=predictions, references=references)["score"])

SacreBLEU score: 46.750469682990165


In [ ]:
predictions = ["This This This This"]
references = [["This plugin allows you to automatically translate web pages between several languages."]]
print("SacreBLEU score:", metric.compute(predictions=predictions, references=references)["score"])

SacreBLEU score: 1.683602693167689


In [ ]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # in case model returns more than pred logits
  if isinstance(preds, tuple):
    preds = preds[0]
  # decode preds
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  # decode labels
  labels = np.where(labels != -100, labels, zokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # post-processing
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions=decoded_predictions, references=decoded_labels)
  return {"blue": result["score"]}

In [ ]:
args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no", # evaluate model once before and after training instead
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# score before fine-tuning
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 21018
  Batch size = 64


In [ ]:
trainer.train()

In [ ]:
# evaluate after fine-tuning
trainer.evaluate(max_length=max_target_length)

In [ ]:
# custom training loop

tokenized_ds.set_format("torch")
train_dl = DataLoader(
    tokenized_ds["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8
)
eval_dl = DataLoader(
    tokenized_ds["validation"],
    collate_fn=data_collator,
    batch_size=8
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
accelerator = Accelerato()
model, optimizer, train_dl, eval_dl = accelerator.prepare(
    model,
    optimizer,
    train_dl,
    eval_dl
)

In [ ]:
num_train_epochs = 3
num_update_steps = num train_epochs * len(train_dl) 
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
def postprocess(predictions, labels):
  predictions = predictions.cpu().numpy()
  labels = labels.cpu().numpy()

  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]
  
  return decoded_preds, decoded_labels

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_training_epochs):
  # training
  model.train()
  for batch in train_dl:
    outputs = model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  # evaluation
  model.eval()
  for batch in tqdm(eval_dl):
    with torch.no_grad():
      generated_tokens = accelerator.unwrap_model(model).generate(
          batch["input_ids"],
          attention_mask=batch["attention_mask"],
          max_langth=128
      )
    labels = batch["labels"]

    generated_tokens = accelerator.pad_across_processes(
            generated_tokens, 
            dim=1, 
            pad_index=tokenizer.pad_token_id
        )
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    predictions_gathered = accelerator.gather(generated_tokens)
    labels_gathered = accelerator.gather(labels)

    decoded_preds, decoded_labels = prostprocess(
        predictions_gathered, 
        labels_gathered
        )
    metric.add_batch(predictions=decoded_preds, references=decoded_labels)
  results = metric.compute()
  print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")